In [ ]:
class TicTacToeBoard:
    def __init__(self):
        self.board = np.zeros(9)
        self.current_player = 1

    def reset(self):
        self.board = np.zeros(9)
        self.current_player = 1

    def make_move(self, move):
        if self.board[move] != 0:
            # Illegal move, game over
            print("Illegal move has been made")
            self.reset()
            return False

        self.board[move] = self.current_player
        self.current_player = -1 if self.current_player == 1 else 1

        return True

    def make_mask(self):

       mask = self.board==0
       return mask

    def is_game_over(self):
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]

        for combination in winning_combinations:
            if self.board[combination[0]] == self.board[combination[1]] == self.board[combination[2]] != 0:
                return True

        return 0 not in self.board

    def get_winner(self):
        winning_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # rows
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # columns
            [0, 4, 8], [2, 4, 6]              # diagonals
        ]

        for combination in winning_combinations:
            if self.board[combination[0]] == self.board[combination[1]] == self.board[combination[2]] != 0:
                return self.board[combination[0]]

        return 0.5

def award_points(result):
    if result == 1:
        return [1.0,-1.0]
    elif result == -1:
        return [-1.0,1.0]
    else:
        return [0.5,0.5]

# Example usage:
board = TicTacToeBoard()
board.make_move(0)  # X plays at position 0
board.make_move(4)  # O plays at position 4
board.make_move(1)  # X plays at position 1
board.make_move(3)  # O plays at position 3
board.make_move(5)  # X plays at position 2

game_over = board.is_game_over()
winner = board.get_winner()

if game_over:
    points = award_points(winner)
    print(f"Game over! Winner: {winner}, Points: {points}")
else:
    print("Game not over yet.")




Game not over yet.


In [ ]:
def random_player(board):
  mask = board.make_mask()
  choices = np.where(mask==1)[0]
  move = np.random.choice(choices)
  #board.make_move(move)
  return move

In [ ]:
import itertools
class Player():
   def __init__(self,board,style,player):
      self.style = style
      self.player = player
      self.board = board
   def make_move(self):
       move = self.player(self.board)
       return move
player_1 = Player(board,style=1.0,player=random_player)
player_2 = Player(board,style =-1.0,player= random_player)

strategies = [player_1,player_2]
strategies = itertools.cycle(strategies)

for games in range(100):
  board.reset()
  count = 0
  while True:
    count+=1
    player = next(strategies)
    #print(player.style)
    move = player.make_move()
    board.make_move(move)
    if(board.is_game_over()):
      print("Game over")
      print(count)
      winner = board.get_winner()
      if(winner==1):
        print("X wins")
      elif(winner==0.5):
        print("Draw")
      else:
        print('Lost')
      reward = award_points(winner)
      break







In [ ]:
a = [1,2]
b = itertools.cycle(a)
for i in range(50):
  print(next(b))

In [ ]:
import torch
import torch.nn as nn

class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits


    def get_action(self, state, legal_moves, temperature):
        logits = self.forward(state)
        masked_logits = logits.clone()
        masked_logits[~legal_moves] = float('-inf')  # Apply mask to illegal moves

        if temperature > 0:
            adjusted_logits = masked_logits / temperature  # Adjust logits with temperature
        else:
            adjusted_logits = masked_logits

        action_probs = torch.softmax(adjusted_logits, dim=0)
        action = torch.multinomial(action_probs, 1).item()
        log_prob = torch.log_softmax(adjusted_logits, dim=0)[action]

        return action, log_prob


In [ ]:
import numpy as np
import torch
import random
board = TicTacToeBoard()
print(board.board)
board.make_move(0)
print(board.board)
board.make_move(1)
mask = board.make_mask().astype(float)
board = random_player(board)
print(board.board)
board = random_player(board)
print(board.board)



[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0.]
[ 1. -1.  1.  0.  0.  0.  0.  0.  0.]
[ 1. -1.  1.  0. -1.  0.  0.  0.  0.]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, legal_moves):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # Apply the mask to illegal moves
        x = x.masked_fill(legal_moves == 0, float('-inf'))

        # Apply softmax activation
        x = F.softmax(x, dim=1)

        return x

# Example usage
input_size = 9  # Size of input state
hidden_size = 128  # Size of hidden layer
output_size = 9  # Size of output action space

# Create an instance of the policy network
policy_net = PolicyNetwork(input_size, hidden_size, output_size)

# Example input state and legal moves mask
state = torch.zeros((1,9))  # Example state (1 x 9)
legal_moves = torch.ones((1,9))  # Example legal moves mask (1 x 9)

# Forward pass through the policy network
probs = policy_net(state, legal_moves)

print("Action Probabilities:", probs)


Action Probabilities: tensor([[0.1059, 0.1273, 0.1254, 0.1162, 0.1128, 0.0904, 0.1219, 0.0925, 0.1076]],
       grad_fn=<SoftmaxBackward0>)
